# Section 506.1 - General
## Building Code
`The floor area of a building shall be determined based on the type of construction, occupancy classification, whether there is an automatic sprinkler system installed throughout the building and the amount of building frontage on public way or open space.`

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

with open("../../../fuseki_endpoints.txt", "r") as file:
    endpoint_url = file.read()
    
# Set up the SPARQL endpoint
sparql = SPARQLWrapper(endpoint_url)

In [2]:
# Getting Construction Type, Occupancy Group, & Sprinkler Type for the building
with open("..\..\..\SPARQL\storeyRule.sparql", "r") as file:
    sparql_query = file.read()

sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Process the results
for result in results["results"]["bindings"]:
    construction_type = result["constructionType"]["value"]
    occupancy_group = result["occupancyGroup"]["value"]
    sprinkler_type = result["sprinklerType"]["value"] if "sprinklerType" in result else "none"
    storey_num = result["storeyNumValue"]["value"]

    # Splitting the construction type and subtype
    if "-" in construction_type:
        main_construction_type, subtype = construction_type.rsplit("-", 1)
    else:
        main_construction_type = construction_type
        subtype = ""

    # Determine the sprinkler system type
    sprinkler_system = "S" if sprinkler_type != "none" else "NS"

    #print(main_construction_type, subtype, occupancy_group, sprinkler_system)

In [3]:
building_frontage = 950.58

print(f"This section is considered: Passed")


print(f"\nThe allowable floor area of the building was determined based on:\n" \
                             f"• the type of construction: {construction_type},\n" \
                             f"• the occupancy classification: {occupancy_group},\n" \
                             f"• the availability of a sprinkler system installed throughout the building: {sprinkler_type},\n" \
                             f"• the amount of building frontage on public way or open space: {building_frontage} ft."
                             f"\n\nPlease refer to the subsections for further analysis results.")

This section is considered: Passed

The allowable floor area of the building was determined based on:
• the type of construction: Type V-A,
• the occupancy classification: R-1,
• the availability of a sprinkler system installed throughout the building: 13,
• the amount of building frontage on public way or open space: 950.58 ft.

Please refer to the subsections for further analysis results.


# Section 506.2 - Allowable area determination
## Building Code
The allowable area of a building shall be determined in accordance with the applicable provisions of Sections 506.2.1 through 506.2.4 and Section 506.3. 

### TABLE 506.2a, b ALLOWABLE AREA FACTOR (At = NS, S1, S13R, or SM, as applicable) IN SQUARE FEET 

In [21]:
# Table 506.2 Display
from IPython.display import display, HTML

rows = []

for occupancy_group, systems in table_506_2.items():
    for sprinkler_system, types in systems.items():
        for construction_type, subtypes in types.items():
            for subtype, area_limit in subtypes.items():
                rows.append({
                    "Occupancy Group": occupancy_group,
                    "Sprinkler System": sprinkler_system,
                    "Construction Type": construction_type,
                    "Subtype": subtype,
                    "Area Limit": area_limit
                })

df = pd.DataFrame(rows)

# Display the DataFrame as a scrollable table
def render_scrollable_table(dataframe, max_height=300):
    display(HTML(f"""
    <div style="max-height: {max_height}px; overflow-y: scroll;">
        {dataframe.to_html(index=False)}
    </div>
    """))

render_scrollable_table(df)

Occupancy Group,Sprinkler System,Construction Type,Subtype,Area Limit
A-1,NS,Type I,A,15500
A-1,NS,Type I,B,8500
A-1,NS,Type II,A,14000
A-1,NS,Type II,B,8500
A-1,NS,Type III,A,15000
A-1,NS,Type III,B,11500
A-1,NS,Type IV,HT,5500
A-1,NS,Type V,A,5500
A-1,S1,Type I,A,62000
A-1,S1,Type I,B,34000


**Note:** UL = Unlimited; NP = Not permitted;  
For SI: 1 square foot = 0.0929 m².  

- **NS** = Buildings not equipped throughout with an automatic sprinkler system  
- **S1** = Buildings a maximum of one story above grade plane equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.1  
- **SM** = Buildings two or more stories above grade plane equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.1  
- **S13R** = Buildings equipped throughout with an automatic sprinkler system installed in accordance with Section 903.3.1.2  

**Footnotes:**

a. See Chapters 4 and 5 for specific exceptions to the allowable height in this chapter.  
b. See Section 903.2 for the minimum thresholds for protection by an automatic sprinkler system for specific occupancies.  
c. New Group H occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.5.  
d. The NS value is only for use in evaluation of existing building area in accordance with the International Existing Building Code.  
e. New Group I-1 and I-3 occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.6. For new Group I-1 occupancies, Condition 1, see Exception 1 of Section 903.2.6.  
f. New and existing Group I-2 occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.6 and Section 1103.5 of the International Fire Code.  
g. New Group I-4 occupancies see Exceptions 2 and 3 of Section 903.2.6.  
h. New Group R occupancies are required to be protected by an automatic sprinkler system in accordance with Section 903.2.8.

In [32]:
with open("..\..\SPARQL\\uniqueOccupancy.sparql", "r") as file:
    sparql_query = file.read()

print(f"This section is considered: Passed"
f"\n\nThe allowable area of the building was determined in accordance with Section 506.2.4 as there is more than one occupancy in the multi-story building: \n")
results = rdf_graph.query(sparql_query)

# Process and format the results
story_dict = {}

for row in results:
    level_value = row[0]
    occupancies = row[1].split(", ")
    
    if level_value not in story_dict:
        story_dict[level_value] = []
    
    story_dict[level_value].extend(occupancies)

# Print the results in the desired format
for level, occupancies in sorted(story_dict.items()):
    unique_occupancies = sorted(set(occupancies))  # Remove duplicates and sort for better display
    num_unique_occupancies = len(unique_occupancies)  # Count the number of unique occupancies
    print(f"•Story: {level} ")
    for occupancy in unique_occupancies:
        print(f"◾Occupancy: {occupancy}")
    print()

This section is considered: Passed

The allowable area of the building was determined in accordance with Section 506.2.4 as there is more than one occupancy in the multi-story building: 

•Story: 01 
◾Occupancy: A-2
◾Occupancy: A-3
◾Occupancy: B
◾Occupancy: R-1

•Story: 02 
◾Occupancy: R-1

•Story: 03 
◾Occupancy: R-1

•Story: 04 
◾Occupancy: R-1



# Section 506.2.1 - Single-occupancy, one-story buildings
## Building Code
```
The allowable area of a single-occupancy building with no more than one story above grade plane shall be determined in accordance with Equation 5-1: 

Aa = At + (NS x If)  (Equation 5-1)

where: 

Aa = Allowable area (square feet). 
At = Tabular allowable area factor (NS, S1, orS13R value, as applicable) in accordance with Table 506.2. 
NS = Tabular allowable area factor in accordancewith Table 506.2 for nonsprinklered building (regardless of whether thebuilding is sprinklered). 
If = Area factor increase due to frontage(percent) as calculated in accordance with Section 506.3. 
```

In [ ]:
if storey_num.lstrip('0') == 1: 
    if num_unique_occupancies == 1: 
        result = "Applicable"
    else:
        result = "Not Applicable"
else: 
    result = "Not Applicable"
    


In [5]:
import rdflib
import pandas as pd
# Load RDF graph
rdf_graph = rdflib.Graph()
rdf_graph.parse("..\..\LBD files\smartReview\outputHotel Compliant Revit 2023 Advanced with Spaces.ifc.ttl", format="ttl")

<Graph identifier=Nd19e33c4365d4a5aa7e18d4b8ae1d402 (<class 'rdflib.graph.Graph'>)>

In [10]:
with open("..\..\SPARQL\\spaceAreaSum.sparql", "r") as file:
    sparql_query = file.read()

results = rdf_graph.query(sparql_query)

# Prepare data for the table
data = {
    "Story": [],
    "Sum": []
}

# Process SPARQL results
for row in results:
    story = str(row["levelValue"]).lstrip('0')  # Remove leading zeros
    area = row["totalArea"]

    data["Story"].append(story)
    data["Sum"].append(area)

df = pd.DataFrame(data)
df

,Story,Sum
0,1,45748.075662204650761
1,3,29796.854063573138566
2,2,29794.857535795623496
3,4,29796.854063573138536
